# Лабораторная работа №3
## Тема: Метод проекции градиента

https://books.google.com.ua/books?id=LLYTCwAAQBAJ&pg=PA699&lpg=PA699&dq=%D0%BF%D1%80%D0%BE%D0%B5%D0%BA%D1%86%D0%B8%D1%8F+%D0%BD%D0%B0+%D1%88%D0%B0%D1%80+%D0%BE%D0%BF%D1%82%D0%B8%D0%BC%D0%B8%D0%B7%D0%B0%D1%86%D0%B8%D1%8F&source=bl&ots=kQxuPkUyWM&sig=ACfU3U2vyEOn3_eeh3FRLAfa6AAkA_e_-A&hl=ru&sa=X&ved=2ahUKEwiI74zKqa7hAhWRposKHdToBRoQ6AEwCXoECAYQAQ#v=onepage&q=%D0%BF%D1%80%D0%BE%D0%B5%D0%BA%D1%86%D0%B8%D1%8F%20%D0%BD%D0%B0%20%D1%88%D0%B0%D1%80%20%D0%BE%D0%BF%D1%82%D0%B8%D0%BC%D0%B8%D0%B7%D0%B0%D1%86%D0%B8%D1%8F&f=false

In [1]:
import sympy as sp
x, y, z, h = sp.symbols('x y z h')

In [2]:
sp.init_printing()

In [3]:
from copy import copy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D
from pprint import pprint

In [4]:
%matplotlib inline
plt.rcParams["figure.figsize"] = (15, 10)

#### Целевая функция
$3x+y+2z \rightarrow min$ 

In [5]:
def f(x):
    y = 3*x[0] + x[1] + 2*x[2]
    return y

#### Условия
$2x^2+y^2+3z^2\le1$

In [6]:
ball_coefs = np.array([2, 1, 3])
ball_center = np.array([0, 0, 0])
ball_radius = 1

#### Производные функции

In [7]:
def df(x, h_step=0.01):
    h_matr = h_step * np.eye(x.shape[0])
    y = np.array([(f(x+h) - f(x-h))/(2*h_step) for h in h_matr])
    return y

In [8]:
def d2f(x, h_step=0.01):
    h_matr = h_step * np.eye(x.shape[0])
    y = np.array([(df(x+h) - df(x-h))/(2*h_step) for h in h_matr])
    return y

#### Функция и ее производные 

In [9]:
a = f(np.array([x, y, z]))
a

In [10]:
b = df(np.array([x, y, z]), h_step=h)
sp.Matrix([sp.simplify(item) for item in b])

⎡3.0⎤
⎢   ⎥
⎢1.0⎥
⎢   ⎥
⎣2.0⎦

In [11]:
c = d2f(np.array([x, y, z]), h_step=h)
sp.Matrix([[sp.simplify(item) for item in row]for row in c])

⎡0  0  0⎤
⎢       ⎥
⎢0  0  0⎥
⎢       ⎥
⎣0  0  0⎦

#### Метод проекции

In [12]:
def ball_proj(x, c, r):
    if np.linalg.norm(x - c) > r:
        proj_x = c + (x - c)/np.linalg.norm(x - c)*r
    else:
        proj_x = x
    return proj_x

#### Градиентный спуск с постоянным шагом

In [13]:
def run_const(start_data, error=1e-6):
    x_arr = copy(start_data)
    alpha = 1
    count = 0
    while True:
        x = x_arr[-1]
        h = df(x)
        x_next = x - alpha * h
        x_prep = np.sqrt(ball_coefs) * x_next
        x_proj = ball_proj(x_prep, ball_center, ball_radius)
        x_arr.append(x_proj)
        if np.linalg.norm(x_proj - x) < error:
            break
        count += 1
        if count == 10**5:
            break
    return x_arr

In [14]:
size = 10
start = [2*size*(np.random.random(3) - 0.5)]
e = 1e-6

In [15]:
first = run_const(start, e)

In [16]:
pprint(first)

[array([-6.79814096,  4.9970572 , -6.40425442]),
 array([-0.67623412,  0.19506474, -0.71039226]),
 array([-0.73734391, -0.11415972, -0.66580141]),
 array([-0.74378632, -0.15678972, -0.64976834]),
 array([-0.74552976, -0.1628896 , -0.64626028]),
 array([-0.74599138, -0.16377366, -0.64550372]),
 array([-0.7461058, -0.1639031, -0.6453386]),
 array([-0.74613306, -0.16392231, -0.6453022 ]),
 array([-0.74613941, -0.16392522, -0.64529412]),
 array([-0.74614087, -0.16392567, -0.64529231]),
 array([-0.74614121, -0.16392575, -0.64529191])]


#### Быстрейший спуск

In [17]:
def run_fastest(start_data, error=1e-6):
    x_arr = copy(start_data)
    alpha = 1
    count = 0
    while True:
        x = x_arr[-1]
        h = df(x)
        A = d2f(x)
        alpha = (h @ h)/(A @ h @ h)
        x_next = x - alpha * h
        x_prep = np.sqrt(ball_coefs) * x_next
        x_proj = ball_proj(x_prep, ball_center, ball_radius)
        x_arr.append(x_proj)
        if np.linalg.norm(x_proj - x) < error:
            break
        count += 1
        if count == 10**5:
            break
    return x_arr

In [18]:
# second = run_fastest(start, e)  # Нулевая матрица вторых производных

In [19]:
# pprint(second)

#### Градиентный спуск с дроблением шага

In [20]:
def run_shredded(start_data, error=1e-6):
    x_arr = copy(start_data)
    coef = 0.5
    alpha= 1
    count = 0
    while True:
        x = x_arr[-1]
        h = df(x)
        while f(x - alpha*h) >= f(x):
            alpha = coef * alpha
        x_next = x - alpha * h
        x_prep = np.sqrt(ball_coefs) * x_next
        x_proj = ball_proj(x_prep, ball_center, ball_radius)
        x_arr.append(x_proj)
        if np.linalg.norm(x_proj - x) < error:
            break
        count += 1
        if count == 10**5:
            break
    return x_arr

In [21]:
third = run_shredded(start, e)

In [22]:
pprint(third)

[array([-6.79814096,  4.9970572 , -6.40425442]),
 array([-0.67623412,  0.19506474, -0.71039226]),
 array([-0.73734391, -0.11415972, -0.66580141]),
 array([-0.74378632, -0.15678972, -0.64976834]),
 array([-0.74552976, -0.1628896 , -0.64626028]),
 array([-0.74599138, -0.16377366, -0.64550372]),
 array([-0.7461058, -0.1639031, -0.6453386]),
 array([-0.74613306, -0.16392231, -0.6453022 ]),
 array([-0.74613941, -0.16392522, -0.64529412]),
 array([-0.74614087, -0.16392567, -0.64529231]),
 array([-0.74614121, -0.16392575, -0.64529191])]
